In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%config InlineBackend.figure_format = 'retina'

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from toy_model import *
import sys 
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR

In [16]:
data_model = reaction_diffusion_equation()
dataset = data_model.genarate_training_data(500, 10, dlt_t = 0.001)

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ParaModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, m_dim):
        super(CustomModel, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim
        
        self.relu_layers = nn.Sequential(*layers)
        
        self.output_layer = nn.Linear(hidden_dims[-1], m_dim)
        self.Q = nn.Parameter(torch.randn(m_dim, m_dim), requires_grad=True)
        
    def forward(self, x):
        x = self.relu_layers(x)
        x = self.output_layer(x)
        x = F.linear(x, self.Q)
        return x

In [18]:
def custom_loss_function(h_outputs, y, lace, mu, lambda1, lambda2):

    g = mu * lace
    
    u_x = y - g
        
    loss1 = 0
    loss2 = 0
    
    k_i_values = torch.zeros(m_dim, device=h_outputs.device)  # 确保k_i_values与h_outputs在同一设备上
    for i in range(m_dim):
        h_i = h_outputs[:, i]
        
        numerator1 = torch.sum(g * h_i) ** 2
        denominator1 = torch.sum(g ** 2) * torch.sum(h_i ** 2)
        
        loss1 += numerator1 / denominator1 if denominator1 != 0 else 0

        numerator2 = torch.sum((y - g) * h_i)
        denominator2 = torch.sqrt(torch.sum(h_i ** 2))
        k_i = numerator2 / denominator2 if denominator2 != 0 else 0
        k_i_values[i] = k_i

    predicted_u_x = torch.matmul(h_outputs, k_i_values.unsqueeze(1)).squeeze()
    loss2 = torch.sum((predicted_u_x - u_x) ** 2)

    predicted_u_x = torch.matmul(h_outputs, k_i_values.unsqueeze(1)).squeeze()
    loss2 = torch.sum((predicted_u_x - u_x) ** 2)
    
    total_loss = lambda1 * loss1 + lambda2 * loss2
    
    return total_loss


In [22]:
dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
for x, y, lace in dataloader:
    break

In [23]:
lace_mean = torch.mean(lace)
y_mean = torch.mean(y)

mu_numerator = torch.sum((lace - lace_mean) * (y - y_mean))
mu_denominator = torch.sum((lace - lace_mean) ** 2)
mu = mu_numerator / mu_denominator

print("Estimated mu:", mu.item())


Estimated mu: -0.2339961677789688


In [20]:
input_dim = 1
hidden_dims = [64, 64, 64]
m_dim = 16